In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy import stats
from yahoo_fin import options as op
from tqdm import tqdm
from datetime import datetime
from dateutil import parser
from joblib import Parallel, delayed
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [93]:
class get_sigma:

    def __init__(self, ticker) -> None:

        yfin = yf.Ticker(ticker)
        da = yfin.history(period = '3mo')
        self.price_list =  list(da.iloc[:,3])
        self.return_list = []
        self.price = self.price_list[-1]

    def calculation(self):

        for i in range(1, len(self.price_list)):
            a = np.log(((self.price_list[i]-self.price_list[i-1])/self.price_list[i-1] + 1))
            self.return_list.append(a)
        self.return_array = self.return_list
        self.mu = np.mean(self.return_array)
        self.sigma = np.sqrt((1 / (len(self.return_list) - 1)) * np.sum(((self.return_array - self.mu) ** 2)))
        self.mu_year = self.mu * 252
        self.sigma_year = self.sigma * np.sqrt(252)


class BlackScholes:

    def __init__(self, s, e, sigma, r, delta_t, dividend=0):

        self.S = s
        self.E = e
        self.sigma = sigma
        self.r = r
        self.delta_t = delta_t
        self.dividend = dividend
        partial = self.r - self.dividend + (1/2)*self.sigma**2
        self.d1 = (np.log(self.S/self.E)+partial*self.delta_t)/(self.sigma*np.sqrt(self.delta_t))
        self.d2 = self.d1 - self.sigma*np.sqrt(self.delta_t)

    def black_scholes(self):

        Part_E = self.E * np.exp(-self.r * self.delta_t) * stats.norm.cdf(self.d2)
        Part_S = self.S * np.exp(-self.dividend * self.delta_t) * stats.norm.cdf(self.d1)
        self.Call = Part_S - Part_E
        self.Put = self.Call - self.S + self.E * np.exp(-self.r * self.delta_t)



class JumpDiffusion:

    """                 dS(t)
                        ----- = mu*dt + sigma*dW(t) + dJ(t)
                        S(t-)

                        S = Current Stock Price
                        E = Strike
                        T = Time to maturity in years
                        σ = Annual Volatility
                        m = Mean of Jump Size
                        v = Standard Deviation of Jump Size
                        λ = Number of jumps per year (intensity)
                        dW(t) = Weiner Process
                        N(t) = Compound Poisson Process
    """

    def __init__(self, s, e, sigma, r, T, m, v, lamb):

        self.s = s
        self.e = e
        self.sigma = sigma
        self.r = r
        self.T = T
        self.m = m
        self.v = v
        self.lam = lamb

    def JD_Call(self):
        call_price = 0
        for k in range(50):
            r_k = self.r - self.lam * (self.m - 1) + (k * np.log(self.m) ) / self.T
            sigma_k = np.sqrt(self.sigma ** 2 + (k * self.v ** 2) / self.T)
            k_fact = np.math.factorial(k)
            bs = BlackScholes(self.s, self.e, sigma_k, r_k, self.T)
            bs.black_scholes()
            self.bs_call = bs.Call
            call_price += (np.exp(- self.m * self.lam * self.T) * (self.m * self.lam * self.T) ** k / (k_fact)) * self.bs_call
        return call_price

    def JD_Put(self):
        put_price = 0
        for k in range(50):
            r_k = self.r - self.lam * (self.m - 1) + (k * np.log(self.m) ) / self.T
            sigma_k = np.sqrt(self.sigma ** 2 + (k * self.v ** 2) / self.T)
            k_fact = np.math.factorial(k)
            bs = BlackScholes(self.s, self.e, sigma_k, r_k, self.T)
            bs.black_scholes()
            self.bs_put = bs.Put
            put_price += (np.exp(- self.m * self.lam * self.T) * (self.m * self.lam * self.T) ** k / (k_fact)) * self.bs_put
        return put_price


def get_ticker():

    sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

    return list(sp500[0]['Symbol'])

def get_stock(ticker):

    data = pd.DataFrame()
    for i in range(len(ticker)):
        yfin = yf.Ticker(ticker[i])
        da = yfin.history(period = '1mo').iloc[-1:,[0,3]]
        da['ticker'] = ticker[i]
        data = pd.concat([data, da])

    return data

def get_call(ticker):

    data = pd.DataFrame()
    ed = op.get_expiration_dates(ticker)

    for i in range(len(ed)):

        call_data = op.get_calls(ticker,ed[i])
        time = (parser.parse(ed[i]).date() - datetime.now().date())
        call_data['Time'] = ed[i]
        call_data['Expiration'] = time.days / 365
        data = pd.concat([data, call_data])

    return data

def get_call_by_ticker(ticker):

    data = pd.DataFrame()
    for i in range(len(ticker)):
        data = pd.concat([data, get_call(ticker[i])])

    return data

def get_full_data(ticker):

    today = datetime.today().strftime('%Y-%m-%d')

    return yf.download(ticker,'2015-01-01',today)[['Open','Close']]

def get_daily_change(tickers):
    data = pd.DataFrame()

    for ticker in tickers:
        df = yf.download(ticker, start='2021-01-01', end=datetime.today().strftime('%Y-%m-%d'))
        df['Daily Change'] = df['Close'].pct_change()
        df = df[['Daily Change']].dropna()
        df.rename(columns={'Daily Change': ticker}, inplace=True)
        data = pd.concat([data, df], axis=1)

    data = data.transpose()
    data.columns = ['day' + str(i + 1) for i in range(len(data.columns))]
    data.insert(0, 'stockname', data.index)

    return data.reset_index(drop=True)

class work:

    def __init__(self, ticker, time_number = 3, Strike_number = 2):

        self.ticker_stock = get_stock([ticker])
        ticker_option = get_call(ticker)
        self.time = ticker_option['Expiration'].unique()[time_number]
        self.option_time = ticker_option[ticker_option['Expiration'] == self.time]
        sigma = get_sigma(ticker)
        sigma.calculation()
        self.sigma = sigma.sigma_year
        self.Strike = Strike_number

    def stupid_simu(self):

        self.call_jd = pd.DataFrame(columns=["lambda", "mean of jump", "variance of jump", "strike", "call_price"])
        results = Parallel(n_jobs=-1)(
            delayed(self.calculate_for_parameters)(lam, m, var, strike)
            for lam in range(8)
            for m in np.arange(0, 1, 0.013)
            for var in np.arange(0, 2, 0.013)
            for strike in [self.option_time['Strike'][self.Strike]]
        )

        for result in results:
            self.call_jd.loc[len(self.call_jd)] = result

        return self.call_jd

    def calculate_for_parameters(self, lam, m, var, strike):

        jump = JumpDiffusion(self.ticker_stock['Open'].item(), strike, self.sigma, 0.045, self.time, np.exp(m+var**2 * 0.5) , var, lam)

        return [lam, m, var, strike, jump.JD_Call()]

    def compare(self):

        real_price = self.option_time[self.option_time.Strike == self.option_time['Strike'][self.Strike]]['Last Price'].item()
        self.call_jd['diff'] = np.abs(self.call_jd.call_price - real_price)
        self.jd = self.call_jd.sort_values(by='diff', ascending=True)

        return self.jd.head(3)

In [ ]:
ticker_list = get_ticker()
X_list = []
Y_list = []

for ticker in tqdm(ticker_list[:2]):
    try:
        a = work(ticker)
        a.stupid_simu()
        results = a.compare()
        results['ticker'] = ticker
        Y_list.append(results)

        daily_change = get_daily_change([ticker])
        daily_change = pd.concat([daily_change]*3, ignore_index=True)
        daily_change['ticker'] = ticker
        X_list.append(daily_change)
    except Exception as e:
        print(f"An error occurred with ticker {ticker}: {e}")

X = pd.concat(X_list).sort_values('ticker')
Y = pd.concat(Y_list).sort_values('ticker')


In [83]:
# Read X and Y data from Excel files
X = pd.read_excel('X.xlsx')
Y = pd.read_excel('Y.xlsx')

# Select the columns in X for input
X_input = X.iloc[:, :-2].dropna()
X_input = X_input.iloc[:, 2:]

# Select the corresponding rows in Y for input
Y_input = Y.iloc[:, [1, 2, 3]].iloc[X_input.index, :]

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_input, Y_input, test_size=0.2)

# Normalize the X data
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled).float()
X_test_tensor = torch.tensor(X_test_scaled).float()
y_train_tensor = torch.tensor(Y_train.values).float()
y_test_tensor = torch.tensor(Y_test.values).float()

# Reshape the tensors for LSTM input
X_train_tensor = X_train_tensor.view(X_train_tensor.shape[0], X_train_tensor.shape[1], 1)
X_test_tensor = X_test_tensor.view(X_test_tensor.shape[0], X_test_tensor.shape[1], 1)
y_train_tensor = y_train_tensor.view(y_train_tensor.shape[0], 3)
y_test_tensor = y_test_tensor.view(y_test_tensor.shape[0], 3)


In [84]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) 
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out


In [87]:
input_size = 1
hidden_size = 50
output_size = 3

model = LSTM(input_size, hidden_size, 1, output_size)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100

for i in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = loss_function(outputs, y_train_tensor)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    optimizer.step()

100%|██████████| 100/100 [03:24<00:00,  2.04s/it]


In [88]:
model.eval()

with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
y_pred_tensor

tensor([[3.6399, 0.3027, 0.5496],
        [3.6392, 0.3031, 0.5511],
        [3.6401, 0.3028, 0.5494],
        [3.6403, 0.3028, 0.5491],
        [3.6402, 0.3030, 0.5495],
        [3.6414, 0.3025, 0.5472],
        [3.6421, 0.3025, 0.5459],
        [3.6391, 0.3029, 0.5509],
        [3.6408, 0.3028, 0.5483],
        [3.6391, 0.3029, 0.5509],
        [3.6396, 0.3030, 0.5504],
        [3.6400, 0.3028, 0.5496],
        [3.6407, 0.3029, 0.5487],
        [3.6425, 0.3028, 0.5458],
        [3.6412, 0.3028, 0.5477],
        [3.6411, 0.3028, 0.5479],
        [3.6395, 0.3030, 0.5504],
        [3.6409, 0.3028, 0.5481],
        [3.6403, 0.3026, 0.5489],
        [3.6400, 0.3028, 0.5497],
        [3.6396, 0.3029, 0.5501],
        [3.6405, 0.3028, 0.5487],
        [3.6403, 0.3029, 0.5493],
        [3.6388, 0.3032, 0.5517],
        [3.6400, 0.3027, 0.5494],
        [3.6403, 0.3028, 0.5491],
        [3.6430, 0.3027, 0.5448],
        [3.6409, 0.3029, 0.5484],
        [3.6408, 0.3029, 0.5486],
        [3.638